In [4]:
import pandas as pd
import numpy as np
import operator

# reading data

# directory = 'instacart_2017_05_01/'
directory = './instacart_2017_05_01/'

print('Loading prior orders')
prior_orders = pd.read_csv(directory + 'order_products__prior.csv', dtype={
        'order_id': np.int32,
        'product_id': np.int32,
        'add_to_cart_order': np.int16,
        'reordered': np.int8}, nrows=652000)

print('Loading orders')
orders = pd.read_csv(directory + 'orders.csv', dtype={
        'order_id': np.int32,
        'user_id': np.int32,
        'eval_set': 'category',
        'order_number': np.int16,
        'order_dow': np.int8,
        'order_hour_of_day': np.int8,
        'days_since_prior_order': np.float32}, nrows=68000)

print('Loading aisles info')
aisles = pd.read_csv(directory + 'products.csv', engine='c',
                           usecols = ['product_id','aisle_id'],
                       dtype={'product_id': np.int32, 'aisle_id': np.int32}, nrows=1000)

pd.set_option('display.float_format', lambda x: '%.3f' % x)

print("\n Checking the loaded CSVs")
print("Prior orders:", prior_orders.shape)
print("Orders", orders.shape)
print("Aisles:", aisles.shape)

Loading prior orders
Loading orders
Loading aisles info

 Checking the loaded CSVs
('Prior orders:', (652000, 4))
('Orders', (68000, 7))
('Aisles:', (1000, 2))


In [5]:
prior_orders.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [6]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,nan
1,2398795,1,prior,2,3,7,15.000
2,473747,1,prior,3,3,12,21.000
3,2254736,1,prior,4,4,7,29.000
4,431534,1,prior,5,4,15,28.000


In [7]:
aisles.head()

,product_id,aisle_id
0,1,61
1,2,104
2,3,94
3,4,38
4,5,5


In [8]:
# removing all user_ids not in the test set from both files to save memory
# the test users present ample data to make models. (and saves space)

test  = orders[orders['eval_set'] == 'test' ]
user_ids = test['user_id'].values
orders = orders[orders['user_id'].isin(user_ids)]

test.shape

(1492, 7)

In [9]:
# Calculate the Prior : p(reordered|product_id)

prior = pd.DataFrame(prior_orders.groupby('product_id')['reordered']     \
                     .agg([('number_of_orders',len),('sum_of_reorders','sum')]))

prior['prior_p'] = (prior['sum_of_reorders']+1)/(prior['number_of_orders']+2) # Informed Prior
# prior['prior_p'] = 1/2  # Flat Prior
# prior.drop(['number_of_orders','sum_of_reorders'], axis=1, inplace=True)

print('Here is The Prior: our first guess of how probable it is that a product be reordered once it has been ordered.')
prior.head()

Here is The Prior: our first guess of how probable it is that a product be reordered once it has been ordered.


,number_of_orders,sum_of_reorders,prior_p
product_id,,,
1,42,29.000,0.682
2,2,0.000,0.250
3,3,2.000,0.600
4,6,2.000,0.375
8,1,0.000,0.333


In [10]:
# merge everything into one dataframe and save any memory space

comb = pd.DataFrame()
comb = pd.merge(prior_orders, orders, on='order_id', how='right')

# slim down comb - 
comb.drop(['eval_set','order_dow','order_hour_of_day'], axis=1, inplace=True)
del prior_orders
del orders

comb = pd.merge(comb, aisles, on ='product_id', how = 'left')
del aisles

prior.reset_index(inplace = True)
comb = pd.merge(comb, prior, on ='product_id', how = 'left')
del prior

print('combined data in DataFrame comb')
comb.head()

combined data in DataFrame comb


,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,days_since_prior_order,aisle_id,number_of_orders,sum_of_reorders,prior_p
0,85,16797.000,1.000,1.000,4041,25,7.000,nan,2844.000,1999.000,0.703
1,85,19691.000,2.000,0.000,4041,25,7.000,nan,253.000,152.000,0.600
2,85,32259.000,3.000,0.000,4041,25,7.000,nan,15.000,7.000,0.471
3,85,20217.000,4.000,0.000,4041,25,7.000,nan,9.000,2.000,0.273
4,85,21349.000,5.000,0.000,4041,25,7.000,nan,13.000,9.000,0.667


In [11]:
recount = pd.DataFrame()
recount['reorder_c'] = comb.groupby(comb.order_id)['reordered'].sum().fillna(0)
bins = [-0.1, 0, 2,4,6,8,11,14,19,71]
cat =  ['None','<=2','<=4','<=6','<=8','<=11','<=14','<=19','>19']
recount['reorder_b'] = pd.cut(recount['reorder_c'], bins, labels = cat)
recount.reset_index(inplace = True)

comb = pd.merge(comb, recount, how = 'left', on = 'order_id')
del recount
comb.head(50)

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,days_since_prior_order,aisle_id,number_of_orders,sum_of_reorders,prior_p,reorder_c,reorder_b
0,85,16797.000,1.000,1.000,4041,25,7.000,nan,2844.000,1999.000,0.703,3.000,<=4
1,85,19691.000,2.000,0.000,4041,25,7.000,nan,253.000,152.000,0.600,3.000,<=4
2,85,32259.000,3.000,0.000,4041,25,7.000,nan,15.000,7.000,0.471,3.000,<=4
3,85,20217.000,4.000,0.000,4041,25,7.000,nan,9.000,2.000,0.273,3.000,<=4
4,85,21349.000,5.000,0.000,4041,25,7.000,nan,13.000,9.000,0.667,3.000,<=4
5,85,33776.000,6.000,0.000,4041,25,7.000,nan,9.000,0.000,0.091,3.000,<=4
6,85,41611.000,7.000,0.000,4041,25,7.000,nan,2.000,1.000,0.500,3.000,<=4
7,85,34063.000,8.000,1.000,4041,25,7.000,nan,38.000,26.000,0.675,3.000,<=4
8,85,42240.000,9.000,1.000,4041,25,7.000,nan,122.000,57.000,0.468,3.000,<=4
9,214,5499.000,1.000,0.000,503,5,9.000,nan,36.000,14.000,0.395,3.000,<=4


In [12]:
bins = [0,2,3,5,7,9,12,17,80]
cat = ['<=2','<=3','<=5','<=7','<=9','<=12','<=17','>17']

comb['atco1'] = pd.cut(comb['add_to_cart_order'], bins, labels = cat)
del comb['add_to_cart_order']
print('comb')
comb.head(50)

comb


,order_id,product_id,reordered,user_id,order_number,days_since_prior_order,aisle_id,number_of_orders,sum_of_reorders,prior_p,reorder_c,reorder_b,atco1
0,85,16797.000,1.000,4041,25,7.000,nan,2844.000,1999.000,0.703,3.000,<=4,<=2
1,85,19691.000,0.000,4041,25,7.000,nan,253.000,152.000,0.600,3.000,<=4,<=2
2,85,32259.000,0.000,4041,25,7.000,nan,15.000,7.000,0.471,3.000,<=4,<=3
3,85,20217.000,0.000,4041,25,7.000,nan,9.000,2.000,0.273,3.000,<=4,<=5
4,85,21349.000,0.000,4041,25,7.000,nan,13.000,9.000,0.667,3.000,<=4,<=5
5,85,33776.000,0.000,4041,25,7.000,nan,9.000,0.000,0.091,3.000,<=4,<=7
6,85,41611.000,0.000,4041,25,7.000,nan,2.000,1.000,0.500,3.000,<=4,<=7
7,85,34063.000,1.000,4041,25,7.000,nan,38.000,26.000,0.675,3.000,<=4,<=9
8,85,42240.000,1.000,4041,25,7.000,nan,122.000,57.000,0.468,3.000,<=4,<=9
9,214,5499.000,0.000,503,5,9.000,nan,36.000,14.000,0.395,3.000,<=4,<=2


In [13]:
atco_fac = pd.DataFrame()
atco_fac = comb.groupby(['reordered', 'atco1'])['atco1'].agg(np.count_nonzero).unstack('atco1')

tot = pd.DataFrame()
tot = np.sum(atco_fac,axis=1)

atco_fac = atco_fac.iloc[:,:].div(tot, axis=0)
atco_fac = atco_fac.stack('atco1')
atco_fac = pd.DataFrame(atco_fac)
atco_fac.reset_index(inplace = True)
atco_fac.rename(columns = {0:'atco_fac_p'}, inplace = True)

comb = pd.merge(comb, atco_fac, how='left', on=('reordered', 'atco1'))
comb.head(50)

,order_id,product_id,reordered,user_id,order_number,days_since_prior_order,aisle_id,number_of_orders,sum_of_reorders,prior_p,reorder_c,reorder_b,atco1,atco_fac_p
0,85,16797.000,1.000,4041,25,7.000,nan,2844.000,1999.000,0.703,3.000,<=4,<=2,0.226
1,85,19691.000,0.000,4041,25,7.000,nan,253.000,152.000,0.600,3.000,<=4,<=2,0.155
2,85,32259.000,0.000,4041,25,7.000,nan,15.000,7.000,0.471,3.000,<=4,<=3,0.077
3,85,20217.000,0.000,4041,25,7.000,nan,9.000,2.000,0.273,3.000,<=4,<=5,0.144
4,85,21349.000,0.000,4041,25,7.000,nan,13.000,9.000,0.667,3.000,<=4,<=5,0.144
5,85,33776.000,0.000,4041,25,7.000,nan,9.000,0.000,0.091,3.000,<=4,<=7,0.134
6,85,41611.000,0.000,4041,25,7.000,nan,2.000,1.000,0.500,3.000,<=4,<=7,0.134
7,85,34063.000,1.000,4041,25,7.000,nan,38.000,26.000,0.675,3.000,<=4,<=9,0.094
8,85,42240.000,1.000,4041,25,7.000,nan,122.000,57.000,0.468,3.000,<=4,<=9,0.094
9,214,5499.000,0.000,503,5,9.000,nan,36.000,14.000,0.395,3.000,<=4,<=2,0.155


In [14]:
aisle_fac = pd.DataFrame()
aisle_fac = comb.groupby(['reordered', 'atco1', 'aisle_id'])['aisle_id']\
                .agg(np.count_nonzero).unstack('aisle_id')

tot = np.sum(aisle_fac,axis=1)

aisle_fac = aisle_fac.iloc[:,:].div(tot, axis=0)
aisle_fac = aisle_fac.stack('aisle_id')
aisle_fac = pd.DataFrame(aisle_fac)
aisle_fac.reset_index(inplace = True)
aisle_fac.rename(columns = {0:'aisle_fac_p'}, inplace = True)

comb = pd.merge(comb, aisle_fac, how = 'left', on = ('aisle_id','reordered','atco1'))
comb.head(50)

,order_id,product_id,reordered,user_id,order_number,days_since_prior_order,aisle_id,number_of_orders,sum_of_reorders,prior_p,reorder_c,reorder_b,atco1,atco_fac_p,aisle_fac_p
0,85,16797.000,1.000,4041,25,7.000,nan,2844.000,1999.000,0.703,3.000,<=4,<=2,0.226,nan
1,85,19691.000,0.000,4041,25,7.000,nan,253.000,152.000,0.600,3.000,<=4,<=2,0.155,nan
2,85,32259.000,0.000,4041,25,7.000,nan,15.000,7.000,0.471,3.000,<=4,<=3,0.077,nan
3,85,20217.000,0.000,4041,25,7.000,nan,9.000,2.000,0.273,3.000,<=4,<=5,0.144,nan
4,85,21349.000,0.000,4041,25,7.000,nan,13.000,9.000,0.667,3.000,<=4,<=5,0.144,nan
5,85,33776.000,0.000,4041,25,7.000,nan,9.000,0.000,0.091,3.000,<=4,<=7,0.134,nan
6,85,41611.000,0.000,4041,25,7.000,nan,2.000,1.000,0.500,3.000,<=4,<=7,0.134,nan
7,85,34063.000,1.000,4041,25,7.000,nan,38.000,26.000,0.675,3.000,<=4,<=9,0.094,nan
8,85,42240.000,1.000,4041,25,7.000,nan,122.000,57.000,0.468,3.000,<=4,<=9,0.094,nan
9,214,5499.000,0.000,503,5,9.000,nan,36.000,14.000,0.395,3.000,<=4,<=2,0.155,nan


In [15]:
recount_fac = pd.DataFrame()
recount_fac = comb.groupby(['reordered', 'atco1', 'reorder_b'])['reorder_b']\
                    .agg(np.count_nonzero).unstack('reorder_b')

tot = pd.DataFrame()
tot = np.sum(recount_fac,axis=1)

recount_fac = recount_fac.iloc[:,:].div(tot, axis=0)
recount_fac.stack('reorder_b')
recount_fac = pd.DataFrame(recount_fac.unstack('reordered').unstack('atco1')).reset_index()
recount_fac.rename(columns = {0:'recount_fac_p'}, inplace = True)

comb = pd.merge(comb, recount_fac, how = 'left', on = ('reorder_b', 'reordered', 'atco1'))
recount_fac.head(50)

,reorder_b,reordered,atco1,recount_fac_p
0,None,0.000,<=2,0.397
1,None,0.000,<=3,0.302
2,None,0.000,<=5,0.270
3,None,0.000,<=7,0.222
4,None,0.000,<=9,0.233
5,None,0.000,<=12,0.199
6,None,0.000,<=17,0.202
7,None,0.000,>17,0.157
8,None,1.000,<=2,nan
9,None,1.000,<=3,nan


In [16]:
p = pd.DataFrame()
p = (comb.loc[:,'atco_fac_p'] * comb.loc[:,'aisle_fac_p'] * comb.loc[:,'recount_fac_p'])
p.reset_index()
comb['p'] = p

comb.head(30)

,order_id,product_id,reordered,user_id,order_number,days_since_prior_order,aisle_id,number_of_orders,sum_of_reorders,prior_p,reorder_c,reorder_b,atco1,atco_fac_p,aisle_fac_p,recount_fac_p,p
0,85,16797.000,1.000,4041,25,7.000,nan,2844.000,1999.000,0.703,3.000,<=4,<=2,0.226,nan,0.219,nan
1,85,19691.000,0.000,4041,25,7.000,nan,253.000,152.000,0.600,3.000,<=4,<=2,0.155,nan,0.142,nan
2,85,32259.000,0.000,4041,25,7.000,nan,15.000,7.000,0.471,3.000,<=4,<=3,0.077,nan,0.208,nan
3,85,20217.000,0.000,4041,25,7.000,nan,9.000,2.000,0.273,3.000,<=4,<=5,0.144,nan,0.274,nan
4,85,21349.000,0.000,4041,25,7.000,nan,13.000,9.000,0.667,3.000,<=4,<=5,0.144,nan,0.274,nan
5,85,33776.000,0.000,4041,25,7.000,nan,9.000,0.000,0.091,3.000,<=4,<=7,0.134,nan,0.226,nan
6,85,41611.000,0.000,4041,25,7.000,nan,2.000,1.000,0.500,3.000,<=4,<=7,0.134,nan,0.226,nan
7,85,34063.000,1.000,4041,25,7.000,nan,38.000,26.000,0.675,3.000,<=4,<=9,0.094,nan,0.075,nan
8,85,42240.000,1.000,4041,25,7.000,nan,122.000,57.000,0.468,3.000,<=4,<=9,0.094,nan,0.075,nan
9,214,5499.000,0.000,503,5,9.000,nan,36.000,14.000,0.395,3.000,<=4,<=2,0.155,nan,0.142,nan


In [19]:
# Calculate bf0 for products when first purchased aka reordered=0
comb0 = pd.DataFrame()
comb0 = comb[comb['reordered']==0]
comb0.loc[:,'first_order'] = comb0['order_number']
# now every product that was ordered has a posterior in usr.
comb0.loc[:,'beta'] = 1
comb0.loc[:,'bf'] = (comb0.loc[:,'prior_p'] * comb0.loc[:,'p']/(1 - comb0.loc[:,'p'])) # bf1
# Small 'slight of hand' here. comb0.bf is really the first posterior and second prior.

# Calculate beta and BF1 for the reordered products
comb1 = pd.DataFrame()
comb1 = comb[comb['reordered']==1]

comb1.loc[:,'beta'] = (1 - .05*comb1.loc[:,'days_since_prior_order']/30)
comb1.loc[:,'bf'] = (1 - comb1.loc[:,'p'])/comb1.loc[:,'p'] # bf0


comb_last = pd.DataFrame()
comb_last = pd.concat([comb0, comb1], axis=0).reset_index(drop=True)

comb_last = comb_last[['reordered', 'user_id', 'order_id', 'product_id','reorder_c','order_number',
                       'bf','beta','atco_fac_p', 'aisle_fac_p', 'recount_fac_p']]
comb_last = comb_last.sort_values((['user_id', 'order_number', 'bf']))

pd.set_option('display.float_format', lambda x: '%.6f' % x)
comb_last.head()

,reordered,user_id,order_id,product_id,reorder_c,order_number,bf,beta,atco_fac_p,aisle_fac_p,recount_fac_p
363,0.000000,36,14400,30415.000000,1.000000,10,nan,1.000000,0.155270,nan,0.304636
364,0.000000,36,14400,1654.000000,1.000000,10,nan,1.000000,0.155270,nan,0.304636
2477,1.000000,36,14400,11079.000000,1.000000,10,nan,0.998333,0.099407,nan,0.082090
1343,0.000000,54,51418,38231.000000,5.000000,29,nan,1.000000,0.144473,nan,0.064057
1344,0.000000,54,51418,16521.000000,5.000000,29,nan,1.000000,0.134190,nan,0.160920


In [20]:
first_order = pd.DataFrame()
first_order = comb_last[comb_last.reordered == 0]
first_order.rename(columns = {'order_number':'first_o'}, inplace = True)
first_order.loc[:,'last_o'] = comb_last.groupby(['user_id'])['order_number'].transform(max)
first_order = first_order[['user_id','product_id','first_o','last_o']]

comb_last = pd.merge(comb_last, first_order, on = ('user_id', 'product_id'), how = 'left')
comb_last.head()

#com = pd.DataFrame()
#com = comb_last[(comb_last.user_id == 3) & (comb_last.first_o < comb_last.order_number)]
#com.groupby([('order_id', 'product_id', 'order_number')])['bf'].agg(np.sum).head(50)

/usr/local/lib/python2.7/dist-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,reordered,user_id,order_id,product_id,reorder_c,order_number,bf,beta,atco_fac_p,aisle_fac_p,recount_fac_p,first_o,last_o
0,0.000000,36,14400,30415.000000,1.000000,10,nan,1.000000,0.155270,nan,0.304636,10.000000,10.000000
1,0.000000,36,14400,1654.000000,1.000000,10,nan,1.000000,0.155270,nan,0.304636,10.000000,10.000000
2,1.000000,36,14400,11079.000000,1.000000,10,nan,0.998333,0.099407,nan,0.082090,nan,nan
3,0.000000,54,51418,38231.000000,5.000000,29,nan,1.000000,0.144473,nan,0.064057,29.000000,29.000000
4,0.000000,54,51418,16521.000000,5.000000,29,nan,1.000000,0.134190,nan,0.160920,29.000000,29.000000


In [21]:
temp = pd.pivot_table(comb_last[(comb_last.user_id == 3) & (comb_last.first_o == comb_last.order_number)],
                     values = 'bf', index = ['user_id', 'product_id'],
                     columns = 'order_number', dropna=False)
temp.head(10)

user_id,product_id


In [22]:
temp = temp.fillna(method='pad', axis=1).fillna(1)
temp.head(10)

user_id,product_id


In [23]:
pd.pivot_table(comb_last[comb_last.first_o <= comb_last.order_number],
                              values = 'bf', index = ['user_id', 'product_id'],
                              columns = 'order_number').head(10)

order_number          1   2   3   4   6   7   8   9   10  11  12  18  21  27  \
user_id product_id                                                             
36      1654.000000  nan nan nan nan nan nan nan nan nan nan nan nan nan nan   
        30415.000000 nan nan nan nan nan nan nan nan nan nan nan nan nan nan   
54      16521.000000 nan nan nan nan nan nan nan nan nan nan nan nan nan nan   
        38231.000000 nan nan nan nan nan nan nan nan nan nan nan nan nan nan   
57      23622.000000 nan nan nan nan nan nan nan nan nan nan nan nan nan nan   
        24852.000000 nan nan nan nan nan nan nan nan nan nan nan nan nan nan   
        35108.000000 nan nan nan nan nan nan nan nan nan nan nan nan nan nan   
68      6649.000000  nan nan nan nan nan nan nan nan nan nan nan nan nan nan   
        8656.000000  nan nan nan nan nan nan nan nan nan nan nan nan nan nan   
        10644.000000 nan nan nan nan nan nan nan nan nan nan nan nan nan nan   

order_number          29  34  43  54  
user_id product_id                    
36      1654.000000  nan nan nan nan  
        30415.000000 nan nan nan nan  
54      16521.000000 nan nan nan nan  
        38231.000000 nan nan nan nan  
57      23622.000000 nan nan nan nan  
        24852.000000 nan nan nan nan  
        35108.000000 nan nan nan nan  
68      6649.000000  nan nan nan nan  
        8656.000000  nan nan nan nan  
        10644.000000 nan nan nan nan

In [24]:
temp.update(pd.pivot_table(comb_last[comb_last.first_o <= comb_last.order_number],
                              values = 'bf', index = ['user_id', 'product_id'],
                              columns = 'order_number'))
temp.head(10)

user_id,product_id


In [25]:
import logging
logging.basicConfig(filename='bayes.log',level=logging.DEBUG)
logging.debug("Started Posterior calculations")
print("Started Posterior calculations")

pred = pd.DataFrame(columns=['user_id', 'product_id'])
# comb_last_temp = pd.DataFrame()
# com = pd.DataFrame()

for uid in comb_last.user_id.unique():
    if uid % 1000 == 0:
        print("Posterior calculated until user %d" % uid)
        logging.debug("Posterior calculated until user %d" % uid)
    
#     del comb_last_temp
    comb_last_temp = pd.DataFrame()
    comb_last_temp = comb_last[comb_last['user_id'] == uid].reset_index()
    
#     del com
    com = pd.DataFrame()
    com = pd.pivot_table(comb_last_temp[comb_last_temp.first_o == comb_last_temp.order_number],
                         values = 'bf', index = ['user_id', 'product_id'],
                         columns = 'order_number', dropna=False)
    com = com.fillna(method='pad', axis=1).fillna(1)
    com.update(pd.pivot_table(comb_last_temp[comb_last_temp.first_o <= comb_last_temp.order_number],
                              values = 'bf', index = ['user_id', 'product_id'],
                              columns = 'order_number'))

    com.reset_index(inplace=True)
    com['posterior'] = com.product(axis=1)
    
    pred = pred.append(com.sort_values(by=['posterior'], ascending=False).head(10)    \
                       .groupby('user_id')['product_id'].apply(list).reset_index())    

print("Posterior calculated for all users")
logging.debug("Posterior calculated for all users")
pred = pred.rename(columns={'product_id': 'products'})
pred.head()

Started Posterior calculations
Posterior calculated for all users


,user_id,products
0,36,"[30415.0, 1654.0]"
0,54,"[38231.0, 16521.0]"
0,57,"[35108.0, 24852.0, 23622.0]"
0,68,"[47547.0, 44359.0, 42445.0, 40377.0, 30233.0, ..."
0,75,"[47626.0, 47042.0, 45200.0, 44837.0, 44683.0, ..."


In [26]:
pred = pred.merge(test, on='user_id', how='left')[['order_id', 'products']]
pred['products'] = pred['products'].apply(lambda x: [int(i) for i in x])    \
                    .astype(str).apply(lambda x: x.strip('[]').replace(',', ''))
pred.head()

,order_id,products
0,1320132,30415 1654
1,1325316,38231 16521
2,320326,35108 24852 23622
3,3024191,47547 44359 42445 40377 30233 23405 21863 2113...
4,1970262,47626 47042 45200 44837 44683 43713 43295 4070...


In [27]:
pred.to_csv('predictions.csv', index=False)